GraphRAG Python Package: Accelerating GenAI With Knowledge Graphs

In [ ]:
#!pip install neo4j-graphrag neo4j openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import openai
import os

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY", "API_KEY"))

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! Can you confirm you're working?"}
    ]
)

print("✅ API key is valid. Response:")
print(response.choices[0].message.content)


✅ API key is valid. Response:
Hello! Yes, I am currently operating and ready to assist you. How can I help you today?


In [ ]:
import os
import asyncio
import neo4j
from dotenv import load_dotenv
from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings as Embeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation.graphrag import GraphRAG

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

driver = neo4j.GraphDatabase.driver(
    "bolt://127.0.0.1:7687",
    auth=("neo4j", "00000000")
)

llm = LLM(
    model_name="gpt-4o",
    model_params={
        "response_format": {"type": "json_object"},
        "temperature": 0
    },
    api_key=OPENAI_KEY
)

embedder = Embeddings(api_key=OPENAI_KEY)

# 1. Build KG and Store in Neo4j Database
kg_builder_pdf = SimpleKGPipeline(
   llm=llm,
   driver=driver,
   embedder=embedder,
   from_pdf=True
)
await kg_builder_pdf.run_async(file_path='C:/Users/ww/Downloads/MR-uniesfahan/PR/sampledata/MIL-PRF-19500_383B.pdf')

# 2. KG Retriever
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder,
   return_properties=["text"],
)


# 3. GraphRAG Class
#llm = LLM(model_name="gpt-4o")
rag = GraphRAG(llm=llm, retriever=vector_retriever)

# 4. Run
response = rag.search( "What is the primary purpose of MIL‑PRF‑19500/383B json?")
print(response.answer)



  {
    "text": "The primary purpose of MIL-PRF-19500/383B is to provide performance specifications for semiconductor devices, specifically silicon voltage-variable capacitors, types 1N5139A through 1N5148A, under the JAN, JANTX, and JANTXV levels. This document outlines the necessary documentation and process conversion measures to comply with the amendment, including specific screening and burn-in conditions, and is approved for use by all Departments and Agencies of the Department of Defense."
  }


In [98]:
questions = [
    "What is the primary purpose of MIL‑PRF‑19500/383B json?",
    "What is the temperature condition specified for hTRB burn‑in testing json?",
    "What is the frequency used for the capacitance test added in Table I, subgroup 4 json?",
    "What is the voltage condition for the capacitance test in Table I, subgroup 4 json?",
    "What does the distribution statement in MIL‑PRF‑19500/383B indicate json?",
    "Which subgroup in Table I had the capacitance test deleted json?",
    "What is the initial value or current limit specified for IR in subgroup 2 of Table I json?",
    "What is the duration specified for hTRB burn‑in testing json?",
    "Which agency is responsible for preparing MIL‑PRF‑19500/383B json?",
    "What is the project number associated with MIL‑PRF‑19500/383B json?"
]

for idx, q in enumerate(questions, start=1):
    try:
        response = rag.search(q)
        print(f"{idx}. Q: {q}")
        print(f"   A: {response.answer}\n")
    except Exception as e:
        print(f"{idx}. Q: {q}")
        print(f"   Error: {e}\n")


1. Q: What is the primary purpose of MIL‑PRF‑19500/383B json?
   A: 
  {
    "text": "The primary purpose of MIL-PRF-19500/383B is to provide performance specifications for semiconductor devices, specifically silicon voltage-variable capacitors, types 1N5139A through 1N5148A, under the JAN, JANTX, and JANTXV levels. This document outlines the necessary documentation and process conversion measures to comply with the amendment, including specific screening and burn-in conditions, and is approved for use by all Departments and Agencies of the Department of Defense."
  }

2. Q: What is the temperature condition specified for hTRB burn‑in testing json?
   A: 
{"TA": "175°C", "VR": "60 V dc", "duration": "96 hours"}

3. Q: What is the frequency used for the capacitance test added in Table I, subgroup 4 json?
   A: 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 

Vector Retriever

In [44]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                   embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

In [45]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder,
   return_properties=["text"],
)


In [47]:
import json

vector_res = vector_retriever.get_search_results(query_text = "What is the primary purpose of MIL‑PRF‑19500/383B?",
             top_k=3)
for i in vector_res.records: print("====n" + json.dumps(i.data(), indent=4))


# output text chunks
'''====
{
    "node": {
        "text": "precise and systematic fashion as suggested here.nFuture care will involve molecular diagnostics throughoutnthe patient timecourse to drive the least toxic combinationnof therapies. Recent evidence suggests a paradigm shift isnon the way but it is hard to predict how fast it will come.nDisclosurenThe authors report no con ufb02icts of interest in this work.nReferencesn1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupusnerythematosus. Lancet .2014 ;384:1878 u20131888. doi:10.1016/S0140-n6736(14)60128"
    },
    "score": 0.9368438720703125
}
====
{
    "node": {
        "text": "d IS agents.nPrecision medicine consists of a tailored approach toneach patient, based on genetic and epigenetic singularities,nwhich in ufb02uence disease pathophysiology and drugnresponse. Precision medicine in SLE is trying to addressnthe need to assess SLE patients optimally, predict diseasencourse and treatment response at diagnosis. Ideally everynpatient would undergo an initial evaluation that wouldnproufb01le his/her disease, assessing the main pathophysiolo-ngic pathway through biomarkers, ther"
    },
    "score": 0.935699462890625
}
====
{
    "node": {
        "text": "REVIEWnT owards Precision Medicine in Systemic LupusnErythematosusnThis article was published in the following Dove Press journal:nPharmacogenomics and Personalized MedicinenElliott Lever1nMarta R Alves2nDavid A Isenberg1n1Centre for Rheumatology, Division ofnMedicine, University College HospitalnLondon, London, UK;2Internal Medicine,nDepartment of Medicine, CentronHospitalar do Porto, Porto, PortugalAbstract: Systemic lupus erythematosus (SLE) is a remarkable condition characterised byndiversit"
    },
    "score": 0.9312744140625
}'''

====n{
    "node": {
        "text": "The documentation and process conversion \nmeasures necessary to comply with this \namendment shall be comtieted bv 15 Julv 2000. \nScreen (see table IV of \nMIL-PRF-19500) \n2 \n10 \nI INCH-POUND I \nMeasurements \nJANTX, JANTXV levels \nTA = +2OO0C, 48 hours \nNot amlicable \nMIL-PRF-l9500/383B \nAMENDMENT 1 \n15 April 2000 \n11 \n12 \n13 \nPERFORMANCE SPECIFICATION SHEET \nIR, \nSee 4.3.1 \nSubgroup 2 of table I herein; \nAIR = 100 percent of initial value or 5 nA dc whichever is \ngreater. \nSEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR \nTYPES 1 N5139A THROUGH 1 N5148A \nJAN, JANTX AND JANTXV \nThis amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and \nis approved for use by all Departments and Agencies of the Department of Defense. \nPAGE 4 \n4.3 Screening table, delete and substitute as follows: \n4.3.1 delete and substitute as follows: \n\u201c4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are a

'====\n{\n    "node": {\n        "text": "precise and systematic fashion as suggested here.nFuture care will involve molecular diagnostics throughoutnthe patient timecourse to drive the least toxic combinationnof therapies. Recent evidence suggests a paradigm shift isnon the way but it is hard to predict how fast it will come.nDisclosurenThe authors report no con ufb02icts of interest in this work.nReferencesn1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupusnerythematosus. Lancet .2014 ;384:1878 u20131888. doi:10.1016/S0140-n6736(14)60128"\n    },\n    "score": 0.9368438720703125\n}\n====\n{\n    "node": {\n        "text": "d IS agents.nPrecision medicine consists of a tailored approach toneach patient, based on genetic and epigenetic singularities,nwhich in ufb02uence disease pathophysiology and drugnresponse. Precision medicine in SLE is trying to addressnthe need to assess SLE patients optimally, predict diseasencourse and treatment response at diagnosis. Ideally everynpatient

In [99]:
for idx, q in enumerate(questions, start=1):
    try:
        res = rag.search(q)
        print(f"{idx}. Q: {q}")
        print(f"   A: {res.answer}\n")
    except Exception as e:
        print(f"{idx}. Q: {q}")
        print(f"   Error: {e}\n")

1. Q: What is the primary purpose of MIL‑PRF‑19500/383B json?
   A: 
  {
    "text": "The primary purpose of MIL-PRF-19500/383B is to provide performance specifications for semiconductor devices, specifically silicon voltage-variable capacitors, types 1N5139A through 1N5148A, under the JAN, JANTX, and JANTXV levels. This document outlines the necessary screening and testing conditions to ensure compliance with military standards, as approved for use by all Departments and Agencies of the Department of Defense."
  }

2. Q: What is the temperature condition specified for hTRB burn‑in testing json?
   A: 
{"TA": "175°C", "VR": "60 V dc", "duration": "96 hours"}

3. Q: What is the frequency used for the capacitance test added in Table I, subgroup 4 json?
   A: 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 

Vector Cypher Retriever

In [4]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder,
   retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks,
 collect(DISTINCT rel) AS rels

//3) format and return context
RETURN '=== text ===n' + apoc.text.join([c in chunks | c.text], 'n---n') + 'nn=== kg_rels ===n' +
 apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], 'n---n') AS info
"""
)

In [50]:
vc_res = vc_retriever.get_search_results(query_text = "What is the primary purpose of MIL‑PRF‑19500/383B?", top_k=3)

# print output
kg_rel_pos = vc_res.records[0]['info'].find('nn=== kg_rels ===n')
print("# Text Chunk Context:")
print(vc_res.records[0]['info'][:kg_rel_pos])
print("# KG Context From Relationships:")
print(vc_res.records[0]['info'][kg_rel_pos:])


# output
# Text Chunk Context:
'''=== text ===
precise and systematic fashion as suggested here.
Future care will involve molecular diagnostics throughout
the patient timecourse to drive the least toxic combination
of therapies. Recent evidence suggests a paradigm shift is
on the way but it is hard to predict how fast it will come.
Disclosure
The authors report no con ﬂicts of interest in this work.
References
1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupus
erythematosus. Lancet .2014 ;384:1878 –1888. doi:10.1016/S0140-
6736(14)60128
---
d IS agents.
Precision medicine consists of a tailored approach to
each patient, based on genetic and epigenetic singularities,
which in ﬂuence disease pathophysiology and drug
response. Precision medicine in SLE is trying to address
the need to assess SLE patients optimally, predict disease
course and treatment response at diagnosis. Ideally every
patient would undergo an initial evaluation that would
proﬁle his/her disease, assessing the main pathophysiolo-
gic pathway through biomarkers, ther
---
REVIEW
T owards Precision Medicine in Systemic Lupus
Erythematosus…


=== kg_rels ===
Systemic lupus erythematosus - AUTHORED(Published in) -> N. Engl. J. Med.
---
Lisnevskaia L - AUTHORED() -> Systemic lupus erythematosus
---
Murphy G - AUTHORED() -> Systemic lupus erythematosus
---
Isenberg DA - AUTHORED() -> Systemic lupus erythematosus
---
Systemic lupus erythematosus - CITES(Published in) -> Lancet
---
Systemic lupus erythematosus - CITES(Systemic lupus erythematosus is discussed in the Lancet publication.) -> Lancet
---
Systemic lupus erythematosus - ASSOCIATED_WITH(SLE is characterized by aberrant activity of the immune system) -> Aberrant activity of the immune system
---
Immunological biomarkers - USED_FOR(Immunological biomarkers could diagnose and monitor disease activity in SLE) -> Systemic lupus erythematosus
---
Novel SLE biomarkers - DESCRIBES(Novel SLE biomarkers have been discovered through omics research) -> Systemic lupus erythematosus
—
…'''

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

'=== text ===\nprecise and systematic fashion as suggested here.\nFuture care will involve molecular diagnostics throughout\nthe patient timecourse to drive the least toxic combination\nof therapies. Recent evidence suggests a paradigm shift is\non the way but it is hard to predict how fast it will come.\nDisclosure\nThe authors report no con ﬂicts of interest in this work.\nReferences\n1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupus\nerythematosus. Lancet .2014 ;384:1878 –1888. doi:10.1016/S0140-\n6736(14)60128\n---\nd IS agents.\nPrecision medicine consists of a tailored approach to\neach patient, based on genetic and epigenetic singularities,\nwhich in ﬂuence disease pathophysiology and drug\nresponse. Precision medicine in SLE is trying to address\nthe need to assess SLE patients optimally, predict disease\ncourse and treatment response at diagnosis. Ideally every\npatient would undergo an initial evaluation that would\nproﬁle his/her disease, assessing the main pathophysio

In [101]:
for idx, q in enumerate(questions, start=1):
    try:
        vc_res = vc_retriever.get_search_results(query_text=q, top_k=3)

        if not vc_res.records:
            print(f"{idx}. Q: {q}")
            print("   No records found.\n")
            continue

        info_text = vc_res.records[0]['info']
        marker = '\n\n=== kg_rels ===\n'
        kg_rel_pos = info_text.find(marker)

        print(f"{idx}. Q: {q}")
        print("# Text Chunk Context:")
        if kg_rel_pos != -1:
            print(info_text[:kg_rel_pos])
            print("# KG Context From Relationships:")
            print(info_text[kg_rel_pos:])
        else:
            # If marker not found, just print all info
            print(info_text)
        print("\n" + "-"*50 + "\n")

    except Exception as e:
        print(f"{idx}. Q: {q}")
        print(f"   Error: {e}\n")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

1. Q: What is the primary purpose of MIL‑PRF‑19500/383B json?
# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB bu

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

2. Q: What is the temperature condition specified for hTRB burn‑in testing json?
# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

...

8. Q: What is the duration specified for hTRB burn‑in testing json?
# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. h

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

9. Q: Which agency is responsible for preparing MIL‑PRF‑19500/383B json?
# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditio

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

10. Q: What is the project number associated with MIL‑PRF‑19500/383B json?
# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in condit

Vector Cypher Retriever

In [ ]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder,
   retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks,
 collect(DISTINCT rel) AS rels

//3) format and return context
RETURN '=== text ===n' + apoc.text.join([c in chunks | c.text], 'n---n') + 'nn=== kg_rels ===n' +
 apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], 'n---n') AS info
"""
)

In [51]:
vc_res = vc_retriever.get_search_results(query_text = "What is the primary purpose of MIL‑PRF‑19500/383B?", top_k=3)

# print output
kg_rel_pos = vc_res.records[0]['info'].find('nn=== kg_rels ===n')
print("# Text Chunk Context:")
print(vc_res.records[0]['info'][:kg_rel_pos])
print("# KG Context From Relationships:")
print(vc_res.records[0]['info'][kg_rel_pos:])


# output
# Text Chunk Context:
'''=== text ===
precise and systematic fashion as suggested here.
Future care will involve molecular diagnostics throughout
the patient timecourse to drive the least toxic combination
of therapies. Recent evidence suggests a paradigm shift is
on the way but it is hard to predict how fast it will come.
Disclosure
The authors report no con ﬂicts of interest in this work.
References
1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupus
erythematosus. Lancet .2014 ;384:1878 –1888. doi:10.1016/S0140-
6736(14)60128
---
d IS agents.
Precision medicine consists of a tailored approach to
each patient, based on genetic and epigenetic singularities,
which in ﬂuence disease pathophysiology and drug
response. Precision medicine in SLE is trying to address
the need to assess SLE patients optimally, predict disease
course and treatment response at diagnosis. Ideally every
patient would undergo an initial evaluation that would
proﬁle his/her disease, assessing the main pathophysiolo-
gic pathway through biomarkers, ther
---
REVIEW
T owards Precision Medicine in Systemic Lupus
Erythematosus…


=== kg_rels ===
Systemic lupus erythematosus - AUTHORED(Published in) -> N. Engl. J. Med.
---
Lisnevskaia L - AUTHORED() -> Systemic lupus erythematosus
---
Murphy G - AUTHORED() -> Systemic lupus erythematosus
---
Isenberg DA - AUTHORED() -> Systemic lupus erythematosus
---
Systemic lupus erythematosus - CITES(Published in) -> Lancet
---
Systemic lupus erythematosus - CITES(Systemic lupus erythematosus is discussed in the Lancet publication.) -> Lancet
---
Systemic lupus erythematosus - ASSOCIATED_WITH(SLE is characterized by aberrant activity of the immune system) -> Aberrant activity of the immune system
---
Immunological biomarkers - USED_FOR(Immunological biomarkers could diagnose and monitor disease activity in SLE) -> Systemic lupus erythematosus
---
Novel SLE biomarkers - DESCRIBES(Novel SLE biomarkers have been discovered through omics research) -> Systemic lupus erythematosus
—
…'''

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

'=== text ===\nprecise and systematic fashion as suggested here.\nFuture care will involve molecular diagnostics throughout\nthe patient timecourse to drive the least toxic combination\nof therapies. Recent evidence suggests a paradigm shift is\non the way but it is hard to predict how fast it will come.\nDisclosure\nThe authors report no con ﬂicts of interest in this work.\nReferences\n1. Lisnevskaia L, Murphy G, Isenberg DA. Systemic lupus\nerythematosus. Lancet .2014 ;384:1878 –1888. doi:10.1016/S0140-\n6736(14)60128\n---\nd IS agents.\nPrecision medicine consists of a tailored approach to\neach patient, based on genetic and epigenetic singularities,\nwhich in ﬂuence disease pathophysiology and drug\nresponse. Precision medicine in SLE is trying to address\nthe need to assess SLE patients optimally, predict disease\ncourse and treatment response at diagnosis. Ideally every\npatient would undergo an initial evaluation that would\nproﬁle his/her disease, assessing the main pathophysio

In [102]:
for idx, q in enumerate(questions, start=1):
    print(f"\n=== {idx}. Q: {q} ===")
    try:
        vc_res = vc_retriever.get_search_results(query_text=q, top_k=3)

        if not vc_res.records:
            print("No records found.")
            continue

        info_text = vc_res.records[0]['info']
        marker = '\n\n=== kg_rels ===\n'
        kg_rel_pos = info_text.find(marker)

        print("# Text Chunk Context:")
        if kg_rel_pos != -1:
            print(info_text[:kg_rel_pos])
            print("# KG Context From Relationships:")
            print(info_text[kg_rel_pos:])
        else:
            # Marker missing → dump whole info
            print(info_text)

    except Exception as e:
        print(f"Error during retrieval: {e}")


=== 1. Q: What is the primary purpose of MIL‑PRF‑19500/383B json? ===


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

...

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

# Text Chunk Context:
=== text ===nThe documentation and process conversion 
measures necessary to comply with this 
amendment shall be comtieted bv 15 Julv 2000. 
Screen (see table IV of 
MIL-PRF-19500) 
2 
10 
I INCH-POUND I 
Measurements 
JANTX, JANTXV levels 
TA = +2OO0C, 48 hours 
Not amlicable 
MIL-PRF-l9500/383B 
AMENDMENT 1 
15 April 2000 
11 
12 
13 
PERFORMANCE SPECIFICATION SHEET 
IR, 
See 4.3.1 
Subgroup 2 of table I herein; 
AIR = 100 percent of initial value or 5 nA dc whichever is 
greater. 
SEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR 
TYPES 1 N5139A THROUGH 1 N5148A 
JAN, JANTX AND JANTXV 
This amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and 
is approved for use by all Departments and Agencies of the Department of Defense. 
PAGE 4 
4.3 Screening table, delete and substitute as follows: 
4.3.1 delete and substitute as follows: 
“4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: 
TA = 175°C; VR = 60 V dc, 96

Instantiate and Run GraphRAG

In [5]:
#from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.generation import RagTemplate
from neo4j_graphrag.generation.graphrag import GraphRAG

#llm = LLM(model_name="gpt-4o",  model_params={"temperature": 0.0})

rag_template = RagTemplate(template='''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned.

# Question:
{query_text}

# Context:
{context}

# Answer:
''', expected_inputs=['query_text', 'context'])

v_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)
vc_rag = GraphRAG(llm=llm, retriever=vc_retriever, prompt_template=rag_template)

In [61]:
q = "Can you summarize the primary purpose of MIL‑PRF‑19500/383B? Provide in detailed list format.json"

v_rag_result = v_rag.search(q, retriever_config={'top_k': 5}, return_context=True)
vc_rag_result = vc_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

print(f"Vector Response: n{v_rag_result.answer}")
print("n===========================n")
print(f"Vector + Cypher Response: n{vc_rag_result.answer}")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

Vector Response: n{
  "primary_purpose_of_MIL‑PRF‑19500/383B": [
    "It is a performance specification sheet for semiconductor devices, specifically silicon diodes and voltage-variable capacitors.",
    "It covers types 1N5139A through 1N5148A.",
    "It includes JAN, JANTX, and JANTXV levels.",
    "The document is approved for use by all Departments and Agencies of the Department of Defense.",
    "It specifies screening and burn-in conditions for these devices.",
    "The amendment includes changes to screening tables and test conditions."
  ]
}
n===========================n
Vector + Cypher Response: n{
  "Primary Purpose of MIL-PRF-19500/383B": [
    "It is a performance specification sheet for semiconductor devices.",
    "Specifically, it covers silicon diodes that function as voltage-variable capacitors.",
    "The types covered are 1N5139A through 1N5148A.",
    "It includes JAN, JANTX, and JANTXV levels.",
    "The document is approved for use by all Departments and Agencies 

In [62]:
for i in v_rag_result.retriever_result.items: print(json.dumps(eval(i.content), indent=1))

{
 "text": "The documentation and process conversion \nmeasures necessary to comply with this \namendment shall be comtieted bv 15 Julv 2000. \nScreen (see table IV of \nMIL-PRF-19500) \n2 \n10 \nI INCH-POUND I \nMeasurements \nJANTX, JANTXV levels \nTA = +2OO0C, 48 hours \nNot amlicable \nMIL-PRF-l9500/383B \nAMENDMENT 1 \n15 April 2000 \n11 \n12 \n13 \nPERFORMANCE SPECIFICATION SHEET \nIR, \nSee 4.3.1 \nSubgroup 2 of table I herein; \nAIR = 100 percent of initial value or 5 nA dc whichever is \ngreater. \nSEMICONDUCTOR DEVICE, DIODE, SILICON, VOLTAGE-VARIABLE CAPACITOR \nTYPES 1 N5139A THROUGH 1 N5148A \nJAN, JANTX AND JANTXV \nThis amendment forms a part of MlL-PRF-l9500/383B, dated 6 August 1999, and \nis approved for use by all Departments and Agencies of the Department of Defense. \nPAGE 4 \n4.3 Screening table, delete and substitute as follows: \n4.3.1 delete and substitute as follows: \n\u201c4.3.1 hTRB burn-in conditions. hTRB burn-in conditions are as follows: \nTA = 175\u00b

In [9]:
questions = [
    "Can you summarize the primary purpose of MIL‑PRF‑19500/383B? Provide in detailed list format.json",
    "What is the temperature condition specified for hTRB burn‑in testing? Provide in detailed list format.json",
    "What is the frequency used for the capacitance test added in Table I, subgroup 4? Provide in detailed list format.json",
    "What is the voltage condition for the capacitance test in Table I, subgroup 4? Provide in detailed list format.json",
    "What does the distribution statement in MIL‑PRF‑19500/383B indicate? Provide in detailed list format.json",
    "Which subgroup in Table I had the capacitance test deleted? Provide in detailed list format.json",
    "What is the initial value or current limit specified for IR in subgroup 2 of Table I? Provide in detailed list format.json",
    "What is the duration specified for hTRB burn‑in testing? Provide in detailed list format.json",
    "Which agency is responsible for preparing MIL‑PRF‑19500/383B? Provide in detailed list format.json",
    "What is the project number associated with MIL‑PRF‑19500/383B? Provide in detailed list format.json"
]

for idx, q in enumerate(questions, start=1):
    print(f"\n=== {idx}. Q: {q} ===")
    try:
        v_rag_result = v_rag.search(q, retriever_config={'top_k': 5}, return_context=True)
        vc_rag_result = vc_rag.search(q, retriever_config={'top_k': 5}, return_context=True)

        print("Vector Response:")
        print(v_rag_result.answer)

        print("\n" + "="*40 + "\n")

        print("Vector + Cypher Response:")
        print(vc_rag_result.answer)
        print("==========================================================")
    except Exception as e:
        print(f"Error processing question: {e}")



=== 1. Q: Can you summarize the primary purpose of MIL‑PRF‑19500/383B? Provide in detailed list format.json ===


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

Vector Response:
{
  "primary_purpose": [
    "MIL-PRF-19500/383B is a performance specification sheet for semiconductor devices.",
    "It specifically covers silicon diodes that function as voltage-variable capacitors.",
    "The types covered are 1N5139A through 1N5148A.",
    "The specification is applicable to JAN, JANTX, and JANTXV levels.",
    "It is approved for use by all Departments and Agencies of the Department of Defense."
  ]
}


Vector + Cypher Response:
{
  "Primary Purpose of MIL-PRF-19500/383B": [
    "It is a performance specification sheet for semiconductor devices, specifically diodes, silicon, voltage-variable capacitors.",
    "It covers types 1N5139A through 1N5148A.",
    "It includes JAN, JANTX, and JANTXV levels.",
    "The document is approved for use by all Departments and Agencies of the Department of Defense.",
    "It specifies hTRB burn-in conditions: TA = 175°C; VR = 60 V dc, 96 hours.",
    "It involves deleting and substituting certain screening tab

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

Vector Response:
{
  "hTRB burn-in conditions": {
    "Temperature (TA)": "175°C",
    "Voltage (VR)": "60 V dc",
    "Duration": "96 hours"
  }
}


Vector + Cypher Response:
{
  "hTRB burn-in conditions": {
    "Temperature": "TA = 175°C",
    "Voltage": "VR = 60 V dc",
    "Duration": "96 hours"
  }
}

=== 3. Q: What is the frequency used for the capacitance test added in Table I, subgroup 4? Provide in detailed list format.json ===


...

Vector Response:
{
  "hTRB burn-in conditions": {
    "Temperature (TA)": "175°C",
    "Voltage (VR)": "60 V dc",
    "Duration": "96 hours"
  }
}


Vector + Cypher Response:
{
  "hTRB burn-in conditions": {
    "Temperature (TA)": "175°C",
    "Voltage (VR)": "60 V dc",
    "Duration": "96 hours"
  }
}

=== 9. Q: Which agency is responsible for preparing MIL‑PRF‑19500/383B? Provide in detailed list format.json ===


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

Vector Response:
{
  "Preparing activity": "DLA - CC"
}


Vector + Cypher Response:
{
  "Preparing activity": "DLA - CC"
}

=== 10. Q: What is the project number associated with MIL‑PRF‑19500/383B? Provide in detailed list format.json ===


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: details)} {position: line: 13, column: 85, offset: 645} for query: "CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k \n//1) Go out 2-3 hops in the entity graph and get relationships\nWITH node AS chunk\nMATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()\nUNWIND relList AS rel\n\n//2) collect relationships and text chunks\nWITH collect(DISTINCT chunk) AS chunks,\n collect(DISTINCT rel) AS rels\n\n//3) format and return context\nRETURN '=== text 

Vector Response:
{
  "Project Number": "5961-2278"
}


Vector + Cypher Response:
{
  "Project Number": "5961-2278"
}


In [10]:
vc_ls = vc_rag_result.retriever_result.items[0].content.split('\n---\n')
for i in vc_ls:
   if "Voltage" in i: print(i)

Retrieval

In [12]:
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
embedder = OpenAIEmbeddings(model="text-embedding-ada-002",api_key=OPENAI_KEY)
retriever = VectorRetriever(
    driver,
    index_name="documentDataEmbedding",
    embedder=embedder,
    return_properties=["title", "plot"],
)

In [ ]:
query_text = "What is the primary purpose of MIL‑PRF‑19500/383B?"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: plot)} {position: line: 1, column: 172, offset: 171} for query: 'CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k RETURN node {.title, .plot} AS node, labels(node) AS nodeLabels, elementId(node) AS elementId, elementId(node) AS id, score'


items=[] metadata={'query_vector': [-0.011870554648339748, -0.0012777803931385279, -0.009062320925295353, 0.00404955493286252, 0.001382919610477984, 0.013152574189007282, -0.01709359884262085, -0.013986905105412006, -0.04045484960079193, -0.013023694045841694, 0.0023486739955842495, 0.005901361349970102, 0.003863017540425062, -0.006586461793631315, -0.027607519179582596, 0.03551669791340828, 0.007671769708395004, -0.002752272877842188, 0.001355786924250424, 0.00782778300344944, -0.0006579678738489747, 0.003065994707867503, -0.02159762568771839, -0.009008055552840233, 0.001872156048193574, 0.014583824202418327, 0.01147034764289856, -0.024297328665852547, -0.00804484449326992, 0.005755523219704628, 0.018911488354206085, 0.0033441048581153154, -0.011375382542610168, -0.019142117351293564, 0.008336520753800869, -0.016469545662403107, 0.010846295394003391, -0.00980846956372261, 0.020363088697195053, -0.013607047498226166, 0.0281366053968668, 0.008818126283586025, 0.025599699467420578, -0.00

GraphRAG

In [69]:
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

# LLM
# Note: the OPENAI_API_KEY must be in the env vars


# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

# Query the graph
query_text = "What is the primary purpose of MIL‑PRF‑19500/383B?json"
response = rag.search(query_text=query_text, retriever_config={"top_k": 5})
print(response.answer)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: plot)} {position: line: 1, column: 172, offset: 171} for query: 'CALL db.index.vector.queryNodes($vector_index_name, $top_k * $effective_search_ratio, $query_vector) YIELD node, score WITH node, score LIMIT $top_k RETURN node {.title, .plot} AS node, labels(node) AS nodeLabels, elementId(node) AS elementId, elementId(node) AS id, score'



{
  "primaryPurpose": "The primary purpose of MIL-PRF-19500/383B is to establish the performance specifications for semiconductor devices, ensuring they meet the necessary quality and reliability standards for military applications."
}


In [14]:
questions = [
    "What is the primary purpose of MIL‑PRF‑19500/383B?json",
    "What is the temperature condition specified for hTRB burn‑in testing?json",
    "What is the frequency used for the capacitance test added in Table I, subgroup 4?json",
    "What is the voltage condition for the capacitance test in Table I, subgroup 4?json",
    "What does the distribution statement in MIL‑PRF‑19500/383B indicate?json",
    "Which subgroup in Table I had the capacitance test deleted?json",
    "What is the initial value or current limit specified for IR in subgroup 2 of Table I?json",
    "What is the duration specified for hTRB burn‑in testing?json",
    "Which agency is responsible for preparing MIL‑PRF‑19500/383B?json",
    "What is the project number associated with MIL‑PRF‑19500/383B?json"
]

for idx, q in enumerate(questions, start=1):
    try:
        response = rag.search(query_text=q, retriever_config={"top_k": 5})
        print(f"{idx}. Q: {q}")
        print(f"   A: {response.answer}\n")
    except Exception as e:
        print(f"{idx}. Q: {q}")
        print(f"   Error: {e}\n")


1. Q: What is the primary purpose of MIL‑PRF‑19500/383B?json
   A: {
  "primary_purpose": "The primary purpose of MIL-PRF-19500/383B is to provide performance specifications for semiconductor devices, specifically silicon voltage-variable capacitors, types 1N5139A through 1N5148A, under the JAN, JANTX, and JANTXV levels. This document outlines the necessary screening and testing conditions to ensure compliance with the specified performance standards for use by all Departments and Agencies of the Department of Defense."
}

2. Q: What is the temperature condition specified for hTRB burn‑in testing?json
   A: 
{"temperature_condition": "TA = 175°C"}

3. Q: What is the frequency used for the capacitance test added in Table I, subgroup 4?json
   A: 
{
  "Answer": "The context does not provide specific information about the frequency used for the capacitance test added in Table I, subgroup 4."
}

4. Q: What is the voltage condition for the capacitance test in Table I, subgroup 4?json
   A: 

RAG With Text2CypherRetriever

In [88]:
from neo4j import GraphDatabase

# Demo database credentials
URI = "neo4j+s://demo.neo4jlabs.com"
AUTH = ("recommendations", "recommendations")

# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM

# Create LLM object


# (Optional) Specify your own Neo4j schema
neo4j_schema = """
  Node properties:
  Person {name: STRING, born: INTEGER}
  Movie {tagline: STRING, title: STRING, released: INTEGER}
  Relationship properties:
  ACTED_IN {roles: LIST}
  REVIEWED {summary: STRING, rating: INTEGER}
  The relationships:
  (:Person)-[:ACTED_IN]->(:Movie)
  (:Person)-[:DIRECTED]->(:Movie)
  (:Person)-[:PRODUCED]->(:Movie)
  (:Person)-[:WROTE]->(:Movie)
  (:Person)-[:FOLLOWS]->(:Person)
  (:Person)-[:REVIEWED]->(:Movie)
"""

# (Optional) Provide user input/query pairs for the LLM to use as examples
examples = [
  "USER INPUT: 'Which actors starred in the Matrix?' QUERY: MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'The Matrix' RETURN p.name"
]

# Initialize the retriever
retriever = Text2CypherRetriever(
  driver=driver,
  llm=llm,
  neo4j_schema=neo4j_schema,
  examples=examples,
)

In [91]:
query_text = "Which movies did Hugo Weaving star in ? json"
print(retriever.search(query_text=query_text))

items=[RetrieverResultItem(content="<Record m.title='Cloud Atlas'>", metadata=None), RetrieverResultItem(content="<Record m.title='The Dressmaker'>", metadata=None), RetrieverResultItem(content="<Record m.title='Babe'>", metadata=None), RetrieverResultItem(content="<Record m.title='V for Vendetta'>", metadata=None), RetrieverResultItem(content="<Record m.title='Matrix, The'>", metadata=None), RetrieverResultItem(content="<Record m.title='Interview, The'>", metadata=None), RetrieverResultItem(content="<Record m.title='Adventures of Priscilla, Queen of the Desert, The'>", metadata=None), RetrieverResultItem(content="<Record m.title='Proof'>", metadata=None)] metadata={'cypher': "MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE p.name = 'Hugo Weaving' RETURN m.title", '__retriever': 'Text2CypherRetriever'}


In [92]:
from neo4j_graphrag.generation import GraphRAG

#llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

# Query the graph
query_text = "Which movies did Hugo Weaving star in?"
response = rag.search(query_text=query_text)
print(response.answer)

Hugo Weaving starred in "V for Vendetta," "The Matrix," "Babe," and "The Adventures of Priscilla, Queen of the Desert."


In [ ]:
# from neo4j_graphrag.retrievers import Text2CypherRetriever
# from neo4j_graphrag.llm import OpenAILLM

# # Create LLM object
# #t2c_llm = OpenAILLM(model_name="gpt-3.5-turbo")

# # (Optional) Specify your own Neo4j schema
# neo4j_schema = """
#   Node properties:
#   Document {code: STRING, data: STRING, createdAt: DATETIME}
#   Chunk {__tmp_internal_id: STRING, data: STRING}
#   Project {code: STRING}

#   The relationships:
#   (:Document)-[:FROM_CHUNK]->(:Chunk)
#   (:Document)-[:HAS_CUSTODIAN]->(:Custodian)
#   (:Document)-[:HAS_PREPARING_ACTIVITY]->(:PreparingActivity)
#   (:Document)-[:HAS_REVIEW_ACTIVITY]->(:ReviewActivity)
#   (:Chunk)-[:PART_OF]->(:Document)
# """


# # (Optional) Provide user input/query pairs for the LLM to use as examples
# # examples = [
# #   "USER INPUT: 'Find document MIL-PRF-19500/383B' "
# #   "QUERY: MATCH (d:Document) WHERE d.code = 'MIL-PRF-19500/383B' RETURN d",

# #   "USER INPUT: 'List chunks in document MIL-PRF-19500/383B' "
# #   "QUERY: MATCH (d:Document)-[:FROM_CHUNK]->(c:Chunk) "
# #   "WHERE d.code = 'MIL-PRF-19500/383B' RETURN c.__tmp_internal_id, c.data"
# # ]



# # Initialize the retriever
# retriever = Text2CypherRetriever(
#   driver=driver,
#   llm=llm,
#   neo4j_schema=neo4j_schema,
#   # examples=examples,
# )

In [93]:
# query_text = "What is the temperature condition specified for hTRB burn‑in testing?in json"
# print(retriever.search(query_text=query_text))

In [94]:
# from neo4j_graphrag.generation import GraphRAG

# # 1) Make sure you actually define an LLM instance — 
# #    Without this, GraphRAG can’t generate Cypher from text.
# #    Uncomment and configure as needed:
# # from neo4j_graphrag.llm import OpenAILLM
# llm = LLM(
#     model_name="gpt-4o",
#     model_params={
#         "temperature": 0
#     },
#     api_key=OPENAI_KEY
# )

# # 2) Initialize the RAG pipeline with the retriever and LLM
# rag = GraphRAG(retriever=retriever, llm=llm)

# # 3) Function to normalize special characters and remove unwanted suffixes
# #    This prevents the LLM from choking on unusual Unicode characters or 
# #    formatting hints like '?json' that aren't part of the actual question.
# def clean_question(q: str) -> str:
#     return (q.strip()
#               .replace('\u2010','-')  # hyphen
#               .replace('\u2011','-')  # non-breaking hyphen
#               .replace('\u2013','-')  # en dash
#               .replace('\u2014','-')  # em dash
#               .replace('\u2212','-')  # minus sign
#               #.removesuffix('?json')  # remove output-format hint
#            )

# # 4) Clean the query before passing it to the retriever
# raw_query = "What is the temperature condition specified for hTRB burn‑in testing?"
# query_text = clean_question(raw_query)

# # 5) Run the query, but first guard against empty LLM output
# #    This prevents sending an empty Cypher string to Neo4j,
# #    which would cause "Unexpected end of input" errors.
# response = rag.search(query_text=query_text+'json')
# if not getattr(response, "answer", "").strip():
#     raise RuntimeError(f"No answer returned for query: {query_text}")

# # 6) Print the retrieved answer
# print(response.answer)
